In [ ]:
!pip install opencv-python tqdm numpy


In [ ]:
import cv2
import os
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Define augmentation functions
def mirror(image):
    """Mirror (Flip) the image horizontally."""
    return cv2.flip(image, 1)

def rotate(image, angle=45):
    """Rotate the image by the given angle."""
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, rotation_matrix, (w, h))

def histogram_equalization(image):
    """Apply histogram equalization to enhance the contrast."""
    if len(image.shape) == 3:  # Check if the image is colored
        ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        y, cr, cb = cv2.split(ycrcb)
        y = cv2.equalizeHist(y)
        ycrcb = cv2.merge((y, cr, cb))
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
    else:  # Grayscale image
        return cv2.equalizeHist(image)

def blur(image, ksize=5):
    """Apply Gaussian Blur to the image."""
    return cv2.GaussianBlur(image, (ksize, ksize), 0)

def sharpen(image):
    """Apply sharpen filter to the image."""
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Augmentation pipeline
def augment_image(image):
    """Apply augmentation transformations."""
    # Path 1: Rotate the original image
    rotated_image = rotate(image)
    rotated_histogram = histogram_equalization(rotated_image)
    rotated_blurred = blur(rotated_image)
    rotated_sharpened = sharpen(rotated_image)

    # Path 2: Mirror then Rotate
    mirrored_image = mirror(image)
    rotated_mirrored_image = rotate(mirrored_image)
    mirrored_rotated_histogram = histogram_equalization(rotated_mirrored_image)
    mirrored_rotated_blurred = blur(rotated_mirrored_image)
    mirrored_rotated_sharpened = sharpen(rotated_mirrored_image)

    return {
        "rotated": rotated_image,
        "rotated_histogram": rotated_histogram,
        "rotated_blurred": rotated_blurred,
        "rotated_sharpened": rotated_sharpened,
        "mirrored_rotated": rotated_mirrored_image,
        "mirrored_rotated_histogram": mirrored_rotated_histogram,
        "mirrored_rotated_blurred": mirrored_rotated_blurred,
        "mirrored_rotated_sharpened": mirrored_rotated_sharpened
    }

# Process dataset
def augment_dataset(input_dir, output_dir):
    """Process and augment images for each class in the dataset."""
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Loop through each class in the dataset
    for class_dir in tqdm(list(input_dir.iterdir()), desc="Processing classes"):
        if class_dir.is_dir():
            class_name = class_dir.name
            class_output_dir = output_dir / class_name
            class_output_dir.mkdir(parents=True, exist_ok=True)

            # Loop through each image in the class folder
            for image_path in class_dir.iterdir():
                if image_path.suffix.lower() in ['.jpg', '.png', '.jpeg']:
                    image = cv2.imread(str(image_path))

                    # Apply augmentations
                    augmented_images = augment_image(image)

                    # Save the original and augmented images
                    cv2.imwrite(str(class_output_dir / image_path.name), image)  # Save original
                    for key, aug_image in augmented_images.items():
                        output_name = f"{image_path.stem}_{key}.jpg"
                        cv2.imwrite(str(class_output_dir / output_name), aug_image)

# Input and output directories
input_dataset_path = "/content/drive/MyDrive/OCTDL"  # Adjust this to your dataset path
output_dataset_path = "/content/drive/MyDrive/OCTDL after augmentation"  # Output for augmented images

# Apply augmentation
augment_dataset(input_dataset_path, output_dataset_path)
print("Augmentation complete. Check the 'augmented_dataset' directory.")


Processing classes: 100%|██████████| 7/7 [05:28<00:00, 46.86s/it]

Augmentation complete. Check the 'augmented_dataset' directory.


In [ ]:
import shutil
shutil.make_archive('augmented_dataset', 'zip', '/content/drive/MyDrive/OCTDL after augmentation')


'/content/augmented_dataset.zip'